## **Setup Rich Text Editor**

In [8]:
from rich.pretty import pprint
from rich import print as fprint
from rich import inspect

## **Load PDF**

In [9]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document

loader: PyPDFLoader = PyPDFLoader(file_path="../assets/NIPS-2017-attention-is-all-you-need-Paper.pdf")

docs: list[Document] = loader.load()
pprint(docs[0])

Document(
│   metadata={
│   │   'producer': 'PyPDF2',
│   │   'creator': 'PyPDF',
│   │   'creationdate': '',
│   │   'subject': 'Neural Information Processing Systems http://nips.cc/',
│   │   'publisher': 'Curran Associates, Inc.',
│   │   'language': 'en-US',
│   │   'created': '2017',
│   │   'eventtype': 'Poster',
│   │   'description-abstract': 'The dominant sequence transduction models are based on complex recurrent orconvolutional neural networks in an encoder and decoder configuration. The best performing such models also connect the encoder and decoder through an attentionm echanisms.  We propose a novel, simple network architecture based solely onan attention mechanism, dispensing with recurrence and convolutions entirely.Experiments on two machine translation tasks show these models to be superiorin quality while being more parallelizable and requiring significantly less timeto train. Our single model with 165 million parameters, achieves 27.5 BLEU onEnglish-to-German translation, improving over the existing best ensemble result by over 1 BLEU. On English-to-French translation, we outperform the previoussingle state-of-the-art with model by 0.7 BLEU, achieving a BLEU score of 41.1.',
│   │   'title': 'Attention is All you Need',
│   │   'date': '2017',
│   │   'moddate': '2018-02-12T21:22:10-08:00',
│   │   'published': '2017',
│   │   'type': 'Conference Proceedings',
│   │   'firstpage': '5998',
│   │   'book': 'Advances in Neural Information Processing Systems 30',
│   │   'description': 'Paper accepted and presented at the Neural Information Processing Systems Conference (http://nips.cc/)',
│   │   'editors': 'I. Guyon and U.V. Luxburg and S. Bengio and H. Wallach and R. Fergus and S. Vishwanathan and R. Garnett',
│   │   'author': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, Illia Polosukhin',
│   │   'lastpage': '6008',
│   │   'source': '../assets/NIPS-2017-attention-is-all-you-need-Paper.pdf',
│   │   'total_pages': 11,
│   │   'page': 0,
│   │   'page_label': '1'
│   },
│   page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser ∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to\nbe superior in quality while being more parallelizable and requiring signiﬁcantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-\nto-German translation task, improving over the existing best results, including\nensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task,\nour model establishes a new single-model state-of-the-art BLEU score of 41.0 after\ntraining for 3.5 days on eight GPUs, a small fraction of the training costs of the\nbest models from the literature.\n1 Introduction\nRecurrent neural networks, long short-term memory [12] and gated recurrent [7] neural networks\nin particular, have been ﬁrmly established as state of the art approaches in sequence modeling and\ntransduction problems such as language modeling and machine translation [ 29, 2, 5]. Numerous\nefforts have since continued to push the boundaries of recurrent language models and encoder-decoder\narchitectures [31, 21, 13].\n∗Equal contributi

## **Semantic Chunking**

In [10]:
!ollama list

NAME                     ID              SIZE      MODIFIED     
llama3.1:8b              46e0c10c039e    4.9 GB    2 hours ago     
nomic-embed-text:v1.5    0a109f422b47    274 MB    26 hours ago    
deepseek-r1:1.5b         e0979632db5a    1.1 GB    7 days ago      
llava:13b                0d0eb4d7f485    8.0 GB    2 months ago    
mistral:latest           3944fe81ec14    4.1 GB    2 months ago    


In [11]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_ollama import  OllamaEmbeddings

embeddings: OllamaEmbeddings = OllamaEmbeddings(model="nomic-embed-text:v1.5")
semantic_splitter: SemanticChunker = SemanticChunker(embeddings=embeddings)

docs_chunks: list[Document] = semantic_splitter.split_documents(documents=docs)
fprint("Length of chunks : ", len(docs_chunks))

Length of chunks :  31

## **Creating Embeddings**

In [12]:
from langchain_chroma.vectorstores import Chroma

vector_store = Chroma.from_documents(
    documents=docs_chunks,
    embedding=embeddings,
    persist_directory="../DB/transformer_paper_db"
)

## **Built in RAG**

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnableLambda

llm = ChatOllama(model="mistral:latest")

prompt = ChatPromptTemplate.from_template(
    """You are a helpful assistant. Use only the context below to answer. If unsure, say "Based on provided info, I don't know."
    Context: {context}
    Question: {input}
    Answer:"""
)

stuff_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt,
) # A Runnable object that will build the prompt with context input. Then llm will create the response.

retrieval_chain = create_retrieval_chain(
    retriever=vector_store.as_retriever(search_kwargs={"k":2}),
    combine_docs_chain=stuff_chain
) # A Runnable Object that will search for similarity and after finding top k similarity pass to stuff chain


main_chain = retrieval_chain | RunnableLambda(lambda x : x.get("answer", ""))

query = "What are transformers?"

for chunk in main_chain.stream({"input":query}):
    print(chunk,end="",flush=False)

 Transformers, based on the provided context, are model architectures that use a multi-layer, multi-head self-attention mechanism and fully connected feed-forward networks as sub-layers. They employ residual connections and layer normalization in each of their layers to facilitate these connections. The dimensions of the output of all sub-layers and embedding layers are set to 512 (dmodel = 512). In the decoder, an additional third sub-layer is added that performs multi-head attention over the output of the encoder stack. To prevent positions from attending to subsequent positions in the decoder stack, self-attention sub-layers are modified with position masking and offset embeddings.